### Introduction

~~According the discussion of yesterday~~
I did a small simulation based on 10 obvervasions and 15 SNP markers using **SNP-BLUP model** (**BayesC0**) but only sample the location parameters

-  chunk 1: Direct solving the MME by invert the coefficient matrix
-  chunk 2: Solving using Gibbs sampler as discripted in Just Jensen's Acta 1994 paper (Also DMU use the way)
-  chunk 3: Gibbs sampler using residual updating

One thing I figured out is that the MCMC chain for the small dataset need be relatively longer than normal, here I used 500000, then I got the two Gibbs sampler (chunk 2 and 3) having the same solusion.

The results from 2 Gibbs sampler were still tiny different compared chunk 1, but is in the second decimal (I think it's acceptable)

### Conclusion

- ~~Gibbs sampler using residual updating has no problem by resulting the same solution,~~ and it is more "programming convenient"
compared with the "Standard updating"


P.S. residual updating method **ONLY** works when the effects are uncorrelated.

In [1]:
# Direct solving the MME
using Distributions
n = 10  # number of observations
k = 15  # number of markers
varg = 15  # total genetic genetic variance
vare = 1.0  # residual variance
vara = varg/k 
λ = vare/vara
srand(123)
M = sample([0,1,2],(n,k))
mu = 10
ONE = ones(n)
mk_effect = rand(Normal(0.0, sqrt(vara)), k)
y = ONE*mu + M*mk_effect + rand(Normal(0.0, sqrt(vare)), n)
lhs = [ONE'ONE ONE'M
    M'ONE M'M + λ*eye(k)]
rhs = [ONE'y
      M'y]
invlhs = inv(lhs)
sol = invlhs*rhs
GEBV =  M*sol[2:end]

10-element Array{Float64,1}:
 -0.964976
 -1.63135 
 -1.10211 
 -4.84306 
  0.642259
  3.0403  
 -7.43588 
 -1.20203 
 -0.810783
 -0.591994

In [2]:
# using gibbs sampling to solve the MME (from Just Jensen's Acta 1994 paper)
mu = mean(y)
#alpha = fill(0.01,k)
sol = fill(0.01,k+1)
inter = 500000
burnin = 200000
step = 50
alphastore = fill(0.0, inter, k)
alphastore2 = fill(0.0, int((inter-burnin)/step + 1), k)
srand(123)
for i=1:inter
    # sampling mu and alpha
    for j=1:k+1
        temp = vec(lhs[j, :])
        splice!(temp, j)
        temp2 = copy(sol)
        splice!(temp2, j)
        me = (rhs[j] - dot(temp, temp2))/lhs[j,j] 
        va = vare/lhs[j,j]
        sol[j] = me + randn()*sqrt(va)
    end
    alphastore[i,:] = sol[2:end]   
end

alphastore2 = alphastore[burnin:step:inter, :] 

alpha=mean(alphastore2, 1)
GEBV4=M*alpha'

10x1 Array{Float64,2}:
 -0.988626
 -1.66735 
 -1.15335 
 -4.8944  
  0.599861
  2.99835 
 -7.49974 
 -1.25973 
 -0.854838
 -0.639961

In [3]:
# Gibbs sampler using residual updating
# BayesC0 for the sample example as upper
function sampleEffects!(nMarkers,xArray,xpx,yCorr,α,vare,varEffects)
    for j=1:nMarkers
        rhs      = dot(xArray[:,j],yCorr) + xpx[j]*α[j]   
        lhs      = xpx[j] + vare/varEffects
        invLhs   = 1.0/lhs
        me       = invLhs*rhs
        oldAlpha = α[j] 
        α[j] = me + randn()*sqrt(invLhs*vare)
        BLAS.axpy!(oldAlpha-α[j],xArray[:,j],yCorr)  
    end
    nothing
end
srand(123)
varEffects = 1.0   # marker variance
vare=1.0
mu = mean(y)
alpha = fill(0.01,k)
nObs = 10
mpm=diag(M'M)
inter = 500000
burnin = 200000
step = 50
alphastore = fill(0.0, inter, k)
alphastore2 = fill(0.0, int((inter-burnin)/step + 1), k)
yCorr = y - mu - M*alpha
for i=1:inter        
        # sample intercept
        yCorr  = yCorr + mu
        rhs    = sum(yCorr)                   
        invLhs = 1.0/(nObs)               
        me     = invLhs*rhs   
        mu     = me + randn()*sqrt(invLhs*vare)
        yCorr  = yCorr - mu                    
        
        # sample effects
        sampleEffects!(k,M,mpm,yCorr,alpha,vare,varEffects)
        alphastore[i,:] = alpha        
end
alphastore2 = alphastore[burnin:step:inter, :] 
alpha=mean(alphastore2, 1)
GEBV3 = M*alpha'

10x1 Array{Float64,2}:
 -0.988626
 -1.66735 
 -1.15335 
 -4.8944  
  0.599861
  2.99835 
 -7.49974 
 -1.25973 
 -0.854838
 -0.639961

In [12]:
dot(vec(lhs[1,:]),vec(alpha))

LoadError: DimensionMismatch("dot product arguments have lengths 16 and 15")
while loading In[12], in expression starting on line 1

In [11]:
alpha

1x15 Array{Float64,2}:
 -0.470992  0.852028  -0.572846  0.221159  …  -0.307005  -1.17163  1.07891